# Import dependencies

In [ ]:
import json, requests
import pandas as pd
from pandas import json_normalize


# Configure Connection Variables

In [ ]:
# Example hostnames: "platform.tetrascience.com" "tetrascience-uat.com"
tdpHostname = "tetrascience-uat.com"
orgSlug = ""
userToken = ""
headers = {"x-org-slug": orgSlug, "ts-auth-token": userToken}
apiRoot = "https://api." + tdpHostname

searchURL = apiRoot + "/v1/datalake/searchEql"
agentURL = apiRoot + "/v1/agents"
labelURL = apiRoot + "/v1/fileinfo/label-fields?from=0&size=1000000"


# Set queries for metadata and labels

In [ ]:
def execute_query(query):
  request = requests.post(searchURL, json=query, headers=headers)
  result = request.json()

  return result

In [ ]:
def generate_query(context_type,field_name):

  if(context_type=="metadata"):
    eqlQuery = {
    "size":0,
    "aggs":{
      "values": {
        "terms":{
          "field":"metadata." + field_name,
          "missing":"Not populated",
          "size":10000
        }
      }
    }
  }
  elif(context_type=="labels"):
    eqlQuery = {
           "size":0,
           "aggs":{
              "labels":{
                 "nested":{
                    "path":"labels"
                 },
                 "aggs":{
                    "filter_label":{
                       "filter":{
                          "bool":{
                             "filter":[
                                {
                                   "term":{
                                      "labels.name":field_name
                                   }
                                }
                             ]
                          }
                       },
                       "aggs":{
                          "values":{
                             "terms":{
                                "field":"labels.value",
                                "missing":"Not populated",
                                "size":10000
                             }
                          }
                       }
                    }
                 }
              }
           }
        }
  else:
        print("Invalid context type\n")

  return eqlQuery

# Function to generate summary and detailed dataframes from EQL result

In [ ]:
# valid context_type values: metadata, labels
def generate_summary_data(context_type):

    individual_dfs = {}
    summary_dict={}
    summary_dict_index=0
    summary_unique_values_dict={}
    summary_unique_values_dict_index=0
    individual_dicts={}
    individual_dict_index=0

    for term in context_terms:
      found_count = 0
      missing_count = 0

      query = generate_query(context_type, term)
      if(context_type == "metadata"):
          result = execute_query(query)['aggregations']['values']['buckets']
      elif(context_type =="labels"):
          result = execute_query(query)
          result_total_files = result['hits']['total']
          result = result['aggregations']['labels']['filter_label']['values']['buckets']
      df = pd.json_normalize(result)
      individual_dfs[term] = df
      individual_dicts[individual_dict_index] = result
      individual_dict_index = individual_dict_index + 1

      for index, row in df.iterrows():
            if(row.key!='Not populated'):
                found_count = found_count + row['doc_count']
                individual_df_index = context_type + "." + term + "." + row.key
                summary_unique_values_dict[summary_unique_values_dict_index] = {'context_type':context_type, 'Key': term, 'Value': row.key, 'Count':row['doc_count']}
                summary_unique_values_dict_index = summary_unique_values_dict_index + 1
            else:
                missing_count = missing_count + row['doc_count']

      if(context_type =="labels"):
        missing_count = result_total_files - found_count

      summary_dict[summary_dict_index] = {'Context':context_type, 'Key':term, 'Found': found_count, 'Missing': missing_count, 'Total': found_count + missing_count}
      summary_dict_index = summary_dict_index + 1

    individual_dfs = pd.DataFrame.from_dict(individual_dicts, orient='index')
    summary_df = pd.DataFrame.from_dict(summary_dict, orient='index')
    summary_unique_values_df = pd.DataFrame.from_dict(summary_unique_values_dict, orient='index')

    return individual_dfs, summary_df, summary_unique_values_df


# Call label listing API

In [ ]:
request = requests.get(labelURL, headers=headers)
label_list_result = request.json()["hits"]

context_terms = []
if(label_list_result):
  for individual_label_result in label_list_result:
    context_terms.append(individual_label_result["name"])

#Display a list of label key names found for target TDP org

---



In [ ]:
context_terms

['barcode',
 'Computer Name',
 'destination',
 'example',
 'Instrument Model',
 'Instrument Vendor',
 'Project',
 'run_fields_assay_finished',
 'run_fields_assay_id',
 'run_fields_barcode',
 'run_fields_column',
 'run_fields_data_link',
 'run_fields_default_flow_rate_mlmin',
 'run_fields_experiment',
 'run_fields_experiment_time',
 'run_fields_export_date_and_time',
 'run_fields_filelink',
 'run_fields_file_link',
 'run_fields_ids_link',
 'run_fields_instrument',
 'run_fields_instrument_name',
 'run_fields_method_name',
 'run_fields_model',
 'run_fields_name',
 'run_fields_plate_type',
 'run_fields_protocol_name',
 'run_fields_rack_id',
 'run_fields_run_datetime',
 'run_fields_sample_id',
 'run_fields_sample_time',
 'run_fields_user',
 'run_fields_vendor',
 'run_processor_name',
 'run_project_id',
 'run_schema_id',
 'signals_experiment',
 'signals_table_template']

# Execute queries and display summary tables

In [ ]:
metadata_individual_dfs, metadata_summary_df, metadata_summary_unique_values_df = generate_summary_data("metadata")
labels_individual_dfs, labels_summary_df, labels_summary_unique_values_df = generate_summary_data("labels")

# Example of content for a summary result dataframe

In [ ]:
summary_merged_result = pd.concat([labels_summary_df, metadata_summary_df])
display(summary_merged_result)

,Context,Key,Found,Missing,Total
0,labels,barcode,3,452,455
1,labels,Computer Name,50,405,455
2,labels,destination,15,440,455
3,labels,example,1,454,455
4,labels,Instrument Model,56,399,455
...,...,...,...,...,...
32,metadata,run_processor_name,0,455,455
33,metadata,run_project_id,0,455,455
34,metadata,run_schema_id,0,455,455
35,metadata,signals_experiment,0,455,455


# Example of content for a detailed result dataframe

In [ ]:
detailed_merged_result = pd.concat([metadata_summary_unique_values_df, labels_summary_unique_values_df])
display(detailed_merged_result)

,context_type,Key,Value,Count
0,labels,barcode,123456,3
1,labels,Computer Name,TS-00001,2
2,labels,Computer Name,TS-00002,2
3,labels,Computer Name,TS-00003,2
4,labels,Computer Name,TS-00004,2
...,...,...,...,...
94,labels,signals_experiment,SIGNALS EXPERIMENT,3
95,labels,signals_experiment,Ian's Pipeline Test Experiment,1
96,labels,signals_table_template,SIGNALS_TABLE_TEMPLATE,2
97,labels,signals_table_template,Lunatic Plate Reader Results,1


# Call Agent list API

In [ ]:
request = requests.get(agentURL, headers=headers)
result = request.json()

# Parse Agent list API response into dataframe summarizing file context

In [ ]:
path_df = pd.DataFrame(columns = ['unique_id_path','fla_id','file_path','path_tags'])

path_df.set_index('unique_id_path', inplace=True)
path_dict={}
dict_index=0

for record in result:
    services_configuration = ""
    file_watcher = ""
    config = ""
    file_paths = ""
    agentID = record['id']
    hostname = record['host']['name']
    config = record.get('config')
    if(config):
        services_configuration=config.get('services_configuration')
    if(services_configuration):
        file_watcher = services_configuration['fileWatcher']
    if(file_watcher):
        file_paths = file_watcher['paths']

    if(file_paths):
        for path in file_paths:
            file_path = path.get('path')
            unique_id_path = agentID + file_path
            path_dict[dict_index] = {'fla_id': agentID, 'hostname':hostname, 'file_path' : file_path, 'path_tags': path.get('tags')}


            metadata_terms = path.get('metadata')
            if(metadata_terms):
                for metadata in metadata_terms:
                    metadata_key = metadata
                    metadata_value = metadata_terms[metadata_key]
                    df_columnname = "metadata." + metadata_key
                    path_dict[dict_index][df_columnname] = metadata_value

            label_terms = path.get('labels')
            if(label_terms):
                for label in label_terms:
                    label_key = label.get('name')
                    label_value = label.get('value')
                    df_columnname = "label." + label_key
                    path_dict[dict_index][df_columnname] = label_value

            dict_index=dict_index+1

path_df = pd.DataFrame.from_dict(path_dict, orient='index')

display(path_df)


,fla_id,hostname,file_path,path_tags,label.Instrument Vendor,label.Instrument Model,label.Lab,label.Instrument Owner,label.Software,label.PC Name,label.Instrument Serial Number
0,61d05b86-3bbb-4e77-adf6-e1e38f716043,TSEMPCLIENTWG,D:\Project-Amplify\instrument1\,None,Vendor1,Model1,High-throughput Screening,Firstname Lastname,Instrument Software v1.0,GHLTBOS1234,DAB123
1,61d05b86-3bbb-4e77-adf6-e1e38f716043,TSEMPCLIENTWG,D:\Project-Amplify\instrument2\,None,Vendor2,Model2,Flow Cytometry,NaN,Instrument Software 2,GHLTBOS5678,SER0123
2,61d05b86-3bbb-4e77-adf6-e1e38f716043,TSEMPCLIENTWG,D:\Project-Amplify\instrument3\,None,Vendor1,Model 3,High-throughput Screening,Firstname Lastname,Instrument Software 3,GHLTBOS9876,SER7493
